In [1]:
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import numpy as np
import pandas as pd

from math import log
from statistics import geometric_mean
import csv
import itertools

py.offline.init_notebook_mode(connected=True)

In [2]:
MIN_NGS_ADDED = 1
BASE = "base"
BASE_HEUR = "base-heur"
TIMEOUT = 900
PROCESSING = "first"

LIMS = {}
LIMS["ground"] = (20, 900)
LIMS["time"] = (0.1, TIMEOUT*1.5)
LIMS["ctime"] = (0.1, TIMEOUT*1.5)


LIMS["csolve"] = (0.1, 900)
LIMS["mem"] = (5_000, 16_800_000)
LIMS["choices"] = (1, 1_000_000_000)
LIMS["conflicts"] = (1, 1_000_000_000)


In [3]:
def get_dom(instance):
	inst = instance.split("/")

	if "ipc-" in instance:
		#return inst[-4] + "-" + inst[-2]
		return "rintanen"

	if "hard" in instance:
		#return inst[-4] + "-" + inst[-3]
		return "hard"
		
	elif "easy" in instance:
		#return inst[-4] + "-" + inst[-3]
		return "easy"
	
	elif "instance" in instance:
		return inst[-3]
	
	elif "asp" in instance:
		return inst[-2]

	raise RuntimeError(f"No dom for instances {instance}")

In [4]:
def read_data(fname, stat_name="value"):
    data = {}
    with open(fname, "r") as _f:
        scalings = []
        
        config_col = []
        instance_col = []
        scaling_col = []
        domain_col = []
        value_col = []
        
        for line in csv.reader(_f, delimiter=","):
            # parse scalings, should happen at the very start
            if line[0] == "scalings":
                scalings = [int(val) for val in line[1:]]
                
            # parse a new config
            # config is at the moment the same, just different horizon
            # so horizon is extracted from it
            elif len(line) == 1 and "learning-" in line[0]:
                # split first element by - and get the last one which should be the horizon number
                config = line[0]

            else:
                domain = line[0].split("/")[0]
                instance = line[0].split("/")[1]
                
                if len(line[0].split("/")) != 2:
                    print("len not 2? instance has / in it!)", instance)
                    print("breaking...")
                    break
                
                #TODO: add row to df with instance, scaling, domain, value of stat
                
                for scaling, val in itertools.zip_longest(scalings, line[1:], fillvalue=""):
                    config_col.append(config)
                    instance_col.append(instance)
                    scaling_col.append(scaling)
                    domain_col.append(domain)
                    value_col.append(val)
    
    data = pd.DataFrame()
    data["config"] = config_col
    data["instance"] = instance_col
    data["scaling"] = scaling_col
    data["domain"] = domain_col
    data[stat_name] = value_col
                
    return data

In [38]:
def check_same_values(df1, df2, columns):
    for name in columns:
        s = df1[name].isin(df2[name]).value_counts()
        if False in s:
            print("Not all values conincide: ", s)

def plotly_scatter_horizons(data, stat, scaling, status, config, min_choices):
          
    # get only the wanted configs
    #pattern = "|".join(configs)
    #print(pattern)
    data = data.groupby("config") #data[data["config"].str.contains(pattern, case=False)]
    data = data.get_group(config)
    
    # get only wanted scaling
    # Can not be multiple because of how we make sure we have a the correct scaling vs scaling 0
    # Since we sort, if we have multiple scalings then the sort will not align the scalings with the base
    # can only have 1 scaling for the sort to work
    # maybe there is a work around it?
    scaling_group = data.groupby("scaling")
    df = scaling_group.get_group(scaling).sort_values(by=["config","domain", "instance"], ignore_index=True, inplace=False)
    
    base_df = scaling_group.get_group(0).sort_values(by=["config","domain", "instance"], ignore_index=True, inplace=False)
    
    # make sure columns have same values
    check_same_values(df, base_df, ["config","domain", "instance"])
    
    print("With given config and horizons\:")
    print("Base statuses: ", base_df["status"].value_counts())
    print("Mod  statuses: ", df["status"].value_counts())
    print()
          
    # get only wanted status
    pattern = "|".join(status)
    print(pattern)
    df = df[df["status"].str.contains(pattern, case=False) & base_df["status"].str.contains(pattern, case=False)]
    base_df = base_df[df["status"].str.contains(pattern, case=False) & base_df["status"].str.contains(pattern, case=False)]
    
    check_same_values(df, base_df, ["config","domain", "instance"])
    
    df = df[base_df["choices"] > min_choices]
    base_df = base_df[base_df["choices"] > min_choices]
    
    layout = go.Layout(
        title=dict(
            text=f"Scatter plot <br> Scaling of {scaling} nogoods <br> {len(df.index)} instances",
            y=0.9,
            x=0.5,
            xanchor="center",
            yanchor="top"
        ),
        yaxis=dict(
            title=f"{stat} with added nogoods",
            type="log",
            range=[log(v,10) for v in LIMS[stat]]
        ),
        xaxis=dict(
            title=f"{stat} with base",
            type="log",
            range=[log(v,10) for v in LIMS[stat]]
        ),
        width=1400,
        height=1000,
    )
    

    df_grouped = df.groupby("domain")
    base_df_grouped = base_df.groupby("domain")

    data = []
    for dom in sorted(set(df["domain"])):
        """
        df = grouped.get_group(dom)
        base_gmean = geometric_mean([float(v)+0.1 for v in df['x']])
        base_avg = df['x'].mean()
        config_gmean = geometric_mean([float(v)+0.1 for v in df['y']])
        config_avg = df['y'].mean()
        ng_avg = df["ngadded"].mean()
        print(f"{stat:10}  :  {dom:50}  :  {len(df.index):4}  :  base : {base_gmean:12.3f} : base avg  : {base_avg:10.2f} config: {config_gmean:12.3f} : config avg  : {config_avg:10.2f}  :  avg ng added: {ng_avg:12.2f}")
        """
        
        df_grouped_dom = df_grouped.get_group(dom).sort_values(by=["config", "instance"], ignore_index=True)
        base_df_grouped_dom = base_df_grouped.get_group(dom).sort_values(by=["config", "instance"], ignore_index=True)
        
        base_gmean = geometric_mean([float(v)+0.1 for v in base_df_grouped_dom[stat]])
        config_gmean = geometric_mean([float(v)+0.1 for v in df_grouped_dom[stat]])

        print(f"{stat:10}  :  {dom:50}  :  {len(df_grouped_dom.index):4}  :  base : {base_gmean:12.3f} : config: {config_gmean:12.3f}")
        
        check_same_values(df_grouped_dom, base_df_grouped_dom, ["config", "instance"])
        #print("With given config and horizons for domain ", dom, " : ")
        #print("Base statuses: ", base_df_grouped_dom["status"].value_counts())
        #print("Mod  statuses: ", df_grouped_dom["status"].value_counts())
        #print()
          
        trace = go.Scatter(
            x=base_df_grouped_dom[stat],
            y=df_grouped_dom[stat],
            mode="markers",
            name=f"{dom}",   
        )
        
        data.append(trace)
        
    fig = go.Figure(data=data, layout=layout)

    if stat in LIMS:
        limits = LIMS[stat]
        fig.add_shape(type="line",x0=0,y0=0, x1=limits[1],y1=limits[1],
                line=dict(color='Red'),
                xref='x',
                yref='y')

    py.offline.iplot(fig)

In [39]:
file_names = {"time": "horizons/asp/results-time.csv",
              "choices": "horizons/asp/results-choices.csv",
              "conflicts": "horizons/asp/results-conflicts.csv",
              "status": "horizons/asp/results-status.csv"
             }

              

In [40]:
DATAFRAMES = {}
FRAME = read_data(file_names["time"], "time")
FRAME.sort_values(by=["config", "domain", "instance", "scaling"], ignore_index=True, inplace=True)
for stat in file_names.keys():
    DATAFRAMES[stat] = read_data(file_names[stat], stat)
    DATAFRAMES[stat].sort_values(by=["config","domain", "instance", "scaling"], ignore_index=True, inplace=True)
    FRAME[stat] = DATAFRAMES[stat][stat]
    
FRAME["choices"] = pd.to_numeric(FRAME["choices"])
FRAME.info()
# do the sorting and then add all stats to ONE frame!!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840 entries, 0 to 3839
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   config     3840 non-null   object
 1   instance   3840 non-null   object
 2   scaling    3840 non-null   int64 
 3   domain     3840 non-null   object
 4   time       3840 non-null   object
 5   choices    3840 non-null   int64 
 6   conflicts  3840 non-null   object
 7   status     3840 non-null   object
dtypes: int64(2), object(6)
memory usage: 240.1+ KB


In [41]:
layout = widgets.Layout(width='400px')

min_choices_widget = widgets.BoundedIntText(min=0, max=50000000, value=0, step=1000, description="Minimum choices")

stats = [s for s in DATAFRAMES.keys()]
stat_widget = widgets.Select(
    options=stats,
    value=stats[0],
    description="Stat"
)


scalings = DATAFRAMES[list(DATAFRAMES.keys())[0]]["scaling"].unique()
scalings = sorted(scalings)[1:]
scaling_widget = widgets.Select(
    options=scalings,
    value=scalings[0],
    description="Scaling"
)

status_widget = widgets.SelectMultiple(
    options=["SATISFIABLE", "UNKNOWN", "UNSATISFIABLE"],
    value=["SATISFIABLE", "UNSATISFIABLE"],
    description="Status"
)

config_list = sorted(list(set(FRAME["config"])))
config_widget = widgets.Select(
    options=config_list,
    value=config_list[0],
    description="Configuration"
)


widgets.interactive(lambda stat, scaling, status, config, min_choices: plotly_scatter_horizons(FRAME, stat, scaling, status, config, min_choices), \
                    stat=stat_widget, scaling=scaling_widget, status=status_widget, config=config_widget, min_choices=min_choices_widget)


interactive(children=(Select(description='Stat', options=('time', 'choices', 'conflicts', 'status'), value='ti…